# 1. Install and Import Dependencies

In [ ]:
!pip install mediapipe opencv-python

In [ ]:
import mediapipe as mp
import cv2
import numpy as np
import uuid
import os

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# 2. Take Output Images 

In [ ]:
import cv2
import mediapipe as mp
import os
import time

# Initialisation of MediaPipe and Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# do not modify parameters
hands = mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) 

# set category of current images
# {zero, one, two, three, four, five, spock}
imageCategory = "spock"

# folder for images
os.makedirs(imageCategory, exist_ok=True) 

cap = cv2.VideoCapture(0)

image_count = 0  

while cap.isOpened() and image_count < 100:  # take 100 images
    ret, frame = cap.read()
    time.sleep(0.5)
    
    # picture gets taken
    #print("Cheese")
    
    # BGR 2 RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Flip on horizontal
    image = cv2.flip(image, 1)
    
    # Set flag
    image.flags.writeable = False
    
    # Detections
    results = hands.process(image)
    
    # Set flag to true
    image.flags.writeable = True
    
    # RGB 2 BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    # create copy of image (before landmarks are drawn) which can be saved later
    image_copy = image.copy()
    
    # Rendering results
    if results.multi_hand_landmarks:
        for num, hand in enumerate(results.multi_hand_landmarks):
            mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                    mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                    mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),
                                     )
    
    # Save the image without landmarks if recognized
    if results.multi_hand_landmarks:
        image_count += 1
        image_filename = f'{imageCategory}/image_{imageCategory}_{image_count}.jpg'
        cv2.imwrite(image_filename, image_copy)
    
    cv2.imshow('Hand Tracking', image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


# 3. Rename And Divide Into Test, Train, Verification

In [4]:
import os
import random
import shutil

# Source and destination folder paths
source_folder =  r"C:\Users\leons\Documents\DeepLearning\DatasetForFingerCounter\ImagesAlex" # Replace this with your source folder path

training_folder = r"C:\Users\leons\Documents\DeepLearning\DatasetForFingerCounter\Training"  # Replace this with your destination folder path
test_folder = r"C:\Users\leons\Documents\DeepLearning\DatasetForFingerCounter\Test"  # Replace this with your destination folder path
verification_folder = r"C:\Users\leons\Documents\DeepLearning\DatasetForFingerCounter\Verification"  # Replace this with your destination folder path

# Prefix to be added
prefix = "Alex"

# Function to move pictures from each subfolder to the destination folder with a new prefix
def move_pictures_with_prefixes(folder_path):
    picture_files = [f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png', '.gif'))]
    random.shuffle(picture_files)

    total_pictures = len(picture_files)
    # 80% are training data
    training_80_percent = int(total_pictures * 0.8)
    # 10% are test data
    test_10_percent = int(total_pictures * 0.1)
    # 10% are verification data
    verification_10_percent = int(total_pictures * 0.1)

    percentages = [training_80_percent, test_10_percent, verification_10_percent, total_pictures]
    folders = [training_folder, test_folder, verification_folder]
    
    for i, percentage in enumerate(percentages[:-1]):
        destination_folder = folders[i]

        selected_files = picture_files[:percentage]
        picture_files = picture_files[percentage:]

        for file_name in selected_files:
            file_path = os.path.join(folder_path, file_name)
            new_file_name = f"{prefix}_{file_name}"
            destination_path = os.path.join(destination_folder, new_file_name)
            shutil.move(file_path, destination_path)

# iterate through all subfolders and do renaming and moving
for folder_name in os.listdir(source_folder):
    current_folder = os.path.join(source_folder, folder_name)
    if os.path.isdir(current_folder):
        move_pictures_with_prefixes(current_folder)
